In [ ]:
# necessary imports 
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Model / data parameters
# Number of classes are 10 from 0-9. Input shape is set to be a 28*28 matrix with just one layer to hold. 
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Lets say if we do not load data directly and plan to load the data as provided to us in columns format then we 
# may need to follow the following approach
import pandas as pd 
from keras.utils.np_utils import to_categorical

df_train = pd.read_csv("../input/digit-recognizer/train.csv")
df_test = pd.read_csv("../input/digit-recognizer/test.csv")

X_train = df_train.drop("label", axis = 1)
Y_train = df_train["label"]

X_test = df_test

X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)

X_train = X_train/255
X_test = X_test/255

Y_train = to_categorical(Y_train, num_classes = 10)

In [ ]:
print("Shape of training dataset{}".format(X_train.shape))
print("Shape of test dataset{}".format(X_test.shape))
print("Shape of Y train dataset{}".format(Y_train.shape))

In [ ]:
print("Shape of training dataset{}".format(x_train.shape))
print("Shape of test dataset{}".format(x_test.shape))
print("Shape of Y train dataset{}".format(y_train.shape))

Max Pooling : It reduces the image size by taking the average across the matrix for 2*2 matrix. Please do refer the link : https://paperswithcode.com/method/max-pooling#:~:text=Max%20Pooling%20is%20a%20pooling,used%20after%20a%20convolutional%20layer.

Flatten : Flatten the entire 2D matrix in a long 1D array. 

In [ ]:
# Simple Covnet Model 
input_shape = (28, 28, 1)
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, Y_train)
print('Train loss: ', train_loss)
print('Train accuracy: ', train_accuracy)

In [ ]:
# predict results
results = model.predict(X_test)

# select the index with the maximum probability
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)